<a href="https://colab.research.google.com/github/MrEplenier/deep-reinforcement-learning/blob/master/Trex_Run_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Mode d'emploi**

Veuillez vous référer au mode d'emploi en préambule du script intitulé *\"Trex Run - Double DQN"*

## Installation et importation des packages 

### Installation

In [ ]:
!pip install opencv-python #Open-CV (traitement image)
!pip install selenium #Selenium (communication avec le navigateur)
!pip install Pillow #Pillow (capture de l'image)
!pip install tensorflow # Tensorflow (Machine-learning)
!pip install Keras # Keras (API Tensorflow)
!pip install matplotlib # Matplotlib (sortie des graphiques)

### Importation

In [ ]:
import cv2
import selenium
import PIL
import time
import matplotlib.pyplot as plt
from PIL import Image, ImageGrab 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np 
from collections import deque
import random
import pickle # sauvegarde de données
import keras
from keras.models import Sequential,load_model
from keras.layers import Conv2D, Activation, Dense, Flatten
from keras.optimizers import Adam

## Paramètres

In [ ]:
discount_rate = 0.95 
epsilon_min = 0.0001 
Epsilon = 0.1 
REPLAY_MEMORY = 30000 
BATCH = 32 
epsilon_decay=0.9995
LEARNING_RATE = 1e-3
ACTIONS = 2 

## Environnement

- Game class: Interface entre le navigateur et python à l'aide de Selenium

In [ ]:
class Game:
    def __init__(self,custom_config=True):
        self._driver = webdriver.Chrome(executable_path ="C:/Users/Thibault/Desktop/Mathieu - RL/chromedriver")
        self._driver.set_window_position(x=-10,y=0) #on positionne la fenetre 
        self._driver.set_window_size(200, 300) #on redimensionne la taille de la fenêtre
        self._driver.get('chrome://dino') #liens de la page du jeu 
        self._driver.execute_script("Runner.config.ACCELERATION=0") #on annule l'accélération initiale du jeu
    
    def get_crashed(self): #récupère le game over
        return self._driver.execute_script("return Runner.instance_.crashed")
    
    def restart(self): #redémarre la partie
        self._driver.execute_script("Runner.instance_.restart()")
        
        time.sleep(0.25)# pas d'actions et d'apprentissage possible pendant 0.25 secondes après le début du jeu
                    
    def press_up(self): # faire sauter l'agent
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    
    def get_score(self):#récuperer le score 
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array) 
        return int(score)

### Dinosaure

In [ ]:
class DinoAgent:
    def __init__(self,game):
        self._game = game; 
        self.jump(); #Saute pour lancer le jeu
        time.sleep(.5) # aucune actions possible pour la premiere fois que le jeu se lance pendant 0.5 secondes
    
    def is_crashed(self): #récupère quand l'agent s'est heurté a un obstacle
        return self._game.get_crashed()
    
    def jump(self): #fait sauter le dinausore 
        self._game.press_up()

- Game state est la fonction principale qui sert de liaison entre l'agent, l'environnement et le code
- get_state(): prends en entrée un array d'action et sert a effectuer les actions de l'agent, retourne le nouvel état, récompenses et si le jeu est terminé.

In [ ]:
class Game_sate:
    def __init__(self,agent,game):  #créer les objets qui vont nous servir dans la suite du code (environnement et agent)
        self._agent = agent 
        self._game = game
        
    def get_state(self,actions):
        score = self._game.get_score()
        is_over = False 
        if actions[0]==1: # ne rien faire
            reward = 1 
        if actions[1] == 1: # sauter
            self._agent.jump()
            reward = -5
        image = grab_screen() # récupération des images après chaque action
        
        if self._agent.is_crashed():
            reward = -100 
            self._game.restart()
            is_over = True #Game over
            
        return image, score, reward, is_over 

## Traitement des images

In [ ]:
def grab_screen(_driver = None):
    screen =  np.array(ImageGrab.grab(bbox=(30,120,450,255))) # Capture l'image dans la zone d'écran décrite dans la bbox
    image = process_img(screen) 
    return image
    
def process_img(image):
    
    # on redimensionne les images
    image = cv2.resize(image, (0,0), fx = 0.50, fy = 0.60) 
    image = image[::2,::2] #on divise le nombre de pixels de l'image par deux 
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200) #permet de déterminer les contours dans l'image en supprimant les détails inutiles et de taille insignifiant (nuage par exemple)
    return  image 

img_rows , img_cols = 105,41 # dimension des images 
img_channels = 4 # nombre d'images par analyse

## Structure du Modèle

Le modèle sera détaillé dans le rapport.

In [ ]:
def buildmodel():
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=4, padding='same',input_shape=(img_cols,img_rows,img_channels)))  #19*40*4
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4), strides=2, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(ACTIONS,activation='linear'))
    model.compile(loss='mse',optimizer=Adam(lr=LEARNING_RATE))
    return model

## Entrainement du modèle

Cette partie sera également détaillé dans le rapport.

In [ ]:
def trainBatch(minibatch,s_t,model):
    
    # lot de quatre images en input
    inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))
    targets = np.zeros((inputs.shape[0], ACTIONS))
    for i in range(0, len(minibatch)):                
        state_t = minibatch[i][0]    
        action_t = minibatch[i][1]   
        reward_t = minibatch[i][2]   
        state_t1 = minibatch[i][3]   
        terminal = minibatch[i][4]   
        inputs[i:i + 1] = state_t    
        targets[i] = model.predict(state_t)  
        Q_sa = model.predict(state_t1)
        if terminal:
            targets[i, action_t] = reward_t 
        else:
            targets[i, action_t] = reward_t + discount_rate * np.max(Q_sa)
    model.train_on_batch(inputs, targets) #equivalent a un model.fit sur les vecteurs  

## Entrainement

In [ ]:
def trainNetwork(model,game_state,D,M):
    # la première action est de ne rien faire lorsque l'on lance l'entrainement
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 #0 => ne rien faire
    
    epsilon=Epsilon
                     
    x_t, scor0 , r_0, terminal = game_state.get_state(do_nothing) # on récupère le résultat de cette première action pour initialiser l'entrainement
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2).reshape(1,41,105,4) # stocke 4 images  redimensionnée en 1*41*105*4 pour initialisé la position

    #on boucle sur 30 millions de partie afin qu'il ne s'arrête pas n'importe quand
    for j in range(30000000):
        action_index = 0
        scor=0 #score 
        r_t = 0 #reward a l'instant t
        while (r_t>=-5): #finit la partie quand la reward est inférieur a -5
    
            a_t = np.zeros([ACTIONS]) # vecteur action à l'instant t     
            
            if  random.random() <= epsilon: #on choisit une action aléatoire si on tire un nombre aléatoire inférieur a epsilon  
                action_index = random.randrange(ACTIONS) #on tire un entier aléatoire dans (0,1)
                a_t[action_index] = 1 # si action_index=0: ne rien faire, si =1 : sauter
                
            
            else: # on prédit à l'aide du modèle quelle serait la "meilleure" action a faire notre instant t
                q = model.predict(s_t)       #on prédit les récompenses associés à chaque action à l'instant t (q_value) 
                action_index = np.argmax(q)         # on prends l'index de la plus grande récompense
                a_t[action_index] = 1        # si action_index=0: ne rien faire, si =1 : sauter
                
            #on execute l'action déterminée précédemment et on regarde ce qu'elle a 
            #produit comme reward ainsi que le nouvel état de notre agent
            x_t1, scor, r_t, terminal = game_state.get_state(a_t)
            
            x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #créer une image de dimension 1*41*105*1
            s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # ajoiute la nouvelle image concernant la position de l'agent en enlevant la plus ancienne des 4 précèdents.

            
            D.append((s_t, action_index, r_t, s_t1, terminal)) # stockage des choix des actions au temps t et ce qu'elles ont entrainée
            
            if len(D) > REPLAY_MEMORY: D.popleft() #on enleve les actions les plus anciennes
               
            s_t = s_t1 #on passe a l'instant t=t+1
        
        #on réduit l'epsilon a chaque fin de partie une fois que le modèle a enregistré 750 tentatives
        if epsilon > epsilon_min and len(M)>750:
            epsilon *= epsilon_decay  
        
        #de même on lance l'apprentissage une fois que le modèle a 750 parties
        if len(M)>750:trainBatch(random.sample(D, BATCH),s_t,model)
        
       
        M.append(scor)
        print("Partie numéro",len(M), "/ EPSILON", "%.2f" % (epsilon*100),"%", "/ score", scor)
        
        #sauvegarde de sécurité des actions tous les 100 due a des erreurs réccurentes de pickle 
        
        if (j+1)% 100 == 0:
            pickle.dump(D, open('C:/Users/Thibault/Desktop/SaveDino/ActionsDino2', 'wb'))
        #Toutes les 30 parties, on sauvegarde notre modèle et nos listes de résultats et d'actions
        
        if (j+1)% 30 == 0:
            #sauvegarde des actions
            pickle.dump(D, open('C:/Users/Thibault/Desktop/SaveDino/ActionsDino', 'wb'))
            #sauvegarde des scores
            pickle.dump(M, open('C:/Users/Thibault/Desktop/SaveDino/ScoreDino', 'wb'))
            #sauvegarde du modele
            model.save('C:/Users/Thibault/Desktop/SaveDino/save.h5')
            
            #on trace un graphique pour voir l'évolution de l'apprentissage
            x=range(len(M))
            y=M
            reg=np.polyfit(x,y,10) #regression polynomiale d'ordre 10 sur l'histogramme d'apprentissage
            reg_coeff=np.poly1d(reg)
            plt.plot(x,y)
            plt.plot(x,reg_coeff(x))
            plt.show()



            


## Lancer un nouvel entrainement

In [ ]:
Epsilon = 0.1 
######## Fonction pour lancer le jeu à 0 #######
def NewGame(observe=False):
    
        # on charge les différentes données de l'environnement (Jeu, Dino, état et rewards)
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)
    
    # on charge la fonction pour avoir un model vierge
    model = buildmodel()
    
    #on initialise les variables contenant les scores et les actions
    D=deque()
    M=[]
    
    #on lance le jeu 
    trainNetwork(model,game_state,D,M)

######### end #########

#on lance le programme 
NewGame(False)

Partie numéro 1 / EPSILON 10.00 % / score 39


## Reprendre un entraînement 

In [ ]:
Epsilon=0.005 #on reprends le epsilon d'ou l'on souhaite repartir

######## Fonction pour relancer le jeu après récupération d'un entrainement #######
def continuer(observe=False):
    
    # on charge les différentes données de l'environnement (Jeu, Dino, état et rewards)
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)
    
    #on télécharge les données sauvegardées lors de l'entrainement précédent:
    
    ##### Les Actions du modèle #####
    D=pickle.load(open('C:/Users/Thibault/Desktop/SaveDino/ActionsDino.', 'rb'))
        
    ##### Les précédents scores pour avoir une continuité dans la courbe #####
    M=pickle.load(open('C:/Users/Thibault/Desktop/SaveDino/ScoreDino', 'rb'))
    
    #### Le modèle qui a déjà commencé son entrainement (on charge les poids déjà présents) #####
    model = load_model('C:/Users/Thibault/Desktop/SaveDino/save.h5')
    
    #on lance le jeu
    trainNetwork(model,game_state,D,M)

############ end ############    
    
#on lance le programme
continuer(False)